# Poll Data - Web Scraping of Real Clear Politics Site

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#using Pandas to get all tables from website page with Poll Data
dfs = pd.read_html('https://www.realclearpolitics.com/epolls/2020/president/us/general_election_trump_vs_biden-6247.html')

dfs

[                             Poll         Date   Sample  MoE  Biden (D)  \
 0                     RCP Average   7/9 - 7/24       --   --       50.0   
 1         CBS News/YouGovCBS News  7/21 - 7/24  1401 LV   --       51.0   
 2      Rasmussen ReportsRasmussen  7/15 - 7/21  2500 LV  2.0       47.0   
 3        The Hill/HarrisXThe Hill  7/17 - 7/20  2829 RV  1.8       45.0   
 4                FOX NewsFOX News  7/12 - 7/15  1104 RV  3.0       49.0   
 5        ABC News/Wash PostABC/WP  7/12 - 7/15   673 LV   --       54.0   
 6  CNBC/Change Research (D)*CNBC*  7/10 - 7/12  1258 LV  2.8       51.0   
 7            QuinnipiacQuinnipiac   7/9 - 7/13  1273 RV  2.8       52.0   
 8   NBC News/Wall St. JrnlNBC/WSJ   7/9 - 7/12   900 RV  3.3       51.0   
 
    Trump (R)      Spread  
 0       40.9  Biden +9.1  
 1       41.0   Biden +10  
 2       45.0    Biden +2  
 3       38.0    Biden +7  
 4       41.0    Biden +8  
 5       44.0   Biden +10  
 6       41.0   Biden +10  
 7       37.0 

In [3]:
# making dfs for each table on the webpage
import urllib
from bs4 import BeautifulSoup

html_table = urllib.request.urlopen('https://www.realclearpolitics.com/epolls/2020/president/us/general_election_trump_vs_biden-6247.html').read()

# fix HTML
soup = BeautifulSoup(html_table, "html.parser")
# warn! id ratings-table is your page specific
for table in soup.findChildren(attrs={'id': 'ratings-table'}): 
    for c in table.children:
        if c.name in ['tbody', 'thead']:
            c.unwrap()

df = pd.read_html(str(soup), flavor="bs4")
len(df[2])

166

In [4]:
#saving df with poll_data needed
poll_data = df[2]

#saving df to csv file
poll_data.to_csv('poll_data.csv')

In [89]:
poll_data_dates = pd.read_csv('data/poll_data_dates.csv')
poll_data_dates = poll_data_dates.dropna()
poll_data_dates['Date'] = pd.to_datetime(poll_data_dates['Date'])

poll_data_dates.sort_values(by=['Date'],ascending=True)
poll_data.isnull().sum()
poll_data_dates.dtypes

Poll                  object
Start Date            object
End Date              object
Date          datetime64[ns]
Sample                object
MoE                   object
Biden (D)            float64
Trump (R)            float64
Spread                object
dtype: object

In [87]:
#Getting Twitter notebook to add poll data

Tweet_notebook = pd.read_csv('data/Tweet_notebook.csv')
Tweet_notebook = Tweet_notebook[['Date','username','Sentiment']]
Tweet_notebook['Date'] = pd.to_datetime(Tweet_notebook['Date'])

Tweet_notebook.isnull().sum()
Tweet_notebook.dtypes

Date         datetime64[ns]
username             object
Sentiment           float64
dtype: object

In [92]:
# merging to get poll data
# In theory, the merge_asof should allow for the poll data to be matched via date to the tweet notebook
# by the closest date. However, I keep getting an error message saying that keys must be sorted. I checked for
# nulls and even sort the keys in the function.

right = poll_data_dates
left = Tweet_notebook
left_sorted = left.sort_values(by="Date")
right_sorted = right.sort_values(by='Date')

poll_df = pd.merge(left_sorted,right_sorted)



In [93]:
poll_df

,Date,username,Sentiment,Poll,Start Date,End Date,Sample,MoE,Biden (D),Trump (R),Spread
0,2020-02-15,_watch_observe_,0.217,NBC News/Wall St. JrnlNBC/WSJ,2/14/20,2/17/20,900 RV,3.3,52.0,44.0,Biden +8
1,2020-02-15,_watch_observe_,0.217,ABC News/Wash PostABC/WP,2/14/20,2/17/20,913 RV,4,52.0,45.0,Biden +7
2,2020-02-17,narniaexpert,0.168,EmersonEmerson,2/16/20,2/18/20,1250 RV,2.7,48.0,52.0,Trump +4
3,2020-02-21,darrylmason,0.000,CBS News/YouGovCBS News,2/20/20,2/22/20,10000 RV,1.2,47.0,45.0,Biden +2
4,2020-02-24,washingtonpost,0.100,IBD/TIPPIBD/TIPP,2/20/20,2/29/20,839 RV,3.5,49.0,46.0,Biden +3
5,2020-02-24,washingtonpost,0.100,FOX NewsFOX News,2/23/20,2/26/20,1000 RV,3,49.0,41.0,Biden +8
6,2020-02-24,washingtonpost,0.100,IBD/TIPPIBD/TIPP,2/20/20,2/29/20,839 RV,3.5,49.0,46.0,Biden +3
7,2020-02-24,washingtonpost,0.100,FOX NewsFOX News,2/23/20,2/26/20,1000 RV,3,49.0,41.0,Biden +8
8,2020-02-24,loyndsview,0.333,IBD/TIPPIBD/TIPP,2/20/20,2/29/20,839 RV,3.5,49.0,46.0,Biden +3
9,2020-02-24,loyndsview,0.333,FOX NewsFOX News,2/23/20,2/26/20,1000 RV,3,49.0,41.0,Biden +8
